In [55]:
import os
import torch
import torch.nn as nn
from model import Transformer
from tokenizeData import Tokenizer

models_folder = "models"
model_name = "alilama_26M"

tok_path = os.path.join(models_folder, "tokenizer.json")
try:
    tokenizer = Tokenizer(path=tok_path)
except:
    raise RuntimeError(f"tokenizer not found. Make sure {tok_path} exists")

device = "cpu"
token_count = len(tokenizer)
model_path = os.path.join(models_folder, model_name+".pth")

if os.path.exists(model_path):
    loaded_state = torch.load(model_path, map_location=device)
    inpt_params = loaded_state["params"]
    inpt_params.max_seq_len = 10000 # increase as seen in ALiBi paper
    model = Transformer(token_count=token_count, device=device, params=inpt_params)
    model.load_state_dict(loaded_state['state_dict'])
    print("Model loaded successfully.")

def generateStream(size, inpt_tokens=None, temperature=1):
    for res in model.genLazy(size, inpt_tokens=inpt_tokens, temperature=temperature):
        l = [res[0][-1]]
        yield tokenizer.detokenize(l)

Model loaded successfully.


In [64]:
# this is the story the model tries to continue:
start_story = 'A long time ago there was a boy named Tim. He liked to ride on his bike.'
# REASONING TEST (sometimes works). We expect something like "come down.":
# start_story = 'If i throw a ball into the air, it will eventually'

modelInput = "STARTSTORY " + start_story
temperature = 2 # how deterministic the model decides the next token (0 = random)
model.eval()
as_tens = tokenizer.tokenize(modelInput, "Tokenize Data", use_tqdm=False)
as_tens = torch.tensor(as_tens).unsqueeze(0).to(device)
for l in generateStream(256, as_tens, temperature=temperature):
    print(l, end="")

 One day, he saw a big tree with a lot of apples. Tim wanted to eat an apple, but he could not reach the apples.
Tim asked his friend, Sam, to help him. Sam said, "I will help you, Tim." Sam was happy to help. He picked the apples and gave them to Tim.
Tim and Sam went to the market to buy apples. They bought apples, bananas, and oranges. They were very happy. But then, something unexpected happened. A big wind came and blew the apples away! Tim and Sam were sad.
But then, a big bird came and picked up the apples. The bird flew away with the apples. Tim and Sam were